In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
DATADIR = "/content/drive/My Drive/data" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_transactions.csv"

In [5]:
data = pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv")

In [6]:
data.head()

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category
0,20337,19666,1,2019-01-01,Покупка,84.00,88676.0,348.0,NaN,Сувениры
1,63404,72991,1,2019-01-01,Покупка,410.00,887248.0,330.0,725.0,Фаст Фуд
2,24789,23517,2,2019-01-01,Покупка,701.44,830014.0,291.0,NaN,Супермаркеты
3,57970,64838,2,2019-01-01,Покупка,6203.70,363834.0,278.0,454.0,Дом/Ремонт
4,12232,11591,2,2019-01-01,Покупка,734.53,85919.0,286.0,878.0,Супермаркеты


To find most popular categories for a client:
1. rank categories within a month
2. apply weughting scheema to months to make more recent dates more relevant
3. multiply category ranks by weights
4. summarize category ranks in all months
5. sort categories by sum of weighted ranks

In [7]:
#convert dates to the begining of the month
data['month_dt']=pd.to_datetime((pd.DatetimeIndex(data['transaction_dttm']).year*10000+pd.DatetimeIndex(data['transaction_dttm']).month*100+1),format='%Y%m%d')

In [9]:
#summarize transactions by month
expenses = pd.DataFrame(data.groupby(['party_rk', 'month_dt', 'category'], as_index=False).transaction_amt_rur.sum())

In [11]:
#sorting transactions within month
expenses = expenses.sort_values(['party_rk','month_dt','transaction_amt_rur'])

In [12]:
#rank categories within month
expenses['cat_rank'] = expenses.groupby(['party_rk','month_dt']).cumcount()+1

In [13]:
#sorting dates for each category
expenses = expenses.sort_values(['party_rk', 'category', 'month_dt'])

In [14]:
#rank dates within category
expenses['month_rank'] = expenses.groupby(['party_rk','category']).cumcount()+1

In [15]:
#assign weights to dates
alpha = 0.8
expenses['weight'] = alpha ** expenses['month_rank']

In [16]:
#calculate weighted rank
expenses['weighted_rank'] = expenses['cat_rank'] * expenses['weight']

In [17]:
#sort categories by weighted rank
top_cat = expenses.groupby(['party_rk', 'category'], as_index=False).weighted_rank.sum().sort_values(['party_rk', 'weighted_rank'], ascending=False)

In [18]:
#find top categories
top_n = 3
top_cat.groupby(['party_rk'], sort=False).head(top_n)

,party_rk,category,weighted_rank
651587,94610,Супермаркеты,7.200000
651584,94610,Одежда/Обувь,7.040000
651580,94610,Аптеки,3.200000
651573,94608,Аптеки,4.800000
651579,94608,Финансовые услуги,4.000000
...,...,...,...
21,5,Супермаркеты,20.647117
8,5,Авиабилеты,18.232115
5,1,Топливо,6.720000
2,1,Одежда/Обувь,4.800000
